# RAG(검색 증강 생성)를 활용한 지능형 Text-to-SQL 시스템 구축

안녕하세요! 이 노트북에서는 RAG(검색 증강 생성) 기술을 활용하여 자연어 질문을 정확한 SQL 쿼리로 변환하는 시스템을 구축해보겠습니다.

## 이 노트북을 통해 배울 수 있는 것
- RAG 기술을 활용한 Text-to-SQL 시스템의 원리와 구현
- 데이터베이스 메타데이터를 벡터 임베딩으로 변환하고 활용하는 방법
- FAISS를 이용한 효율적인 벡터 검색 구현
- Amazon Bedrock을 활용한 임베딩 및 LLM 연동
- 실제 TPC-DS 데이터셋을 대상으로 한 실습

## 배경 지식
RAG(Retrieval-Augmented Generation)는 기존 지식 베이스에서 관련 정보를 검색하여 AI 모델의 답변 품질을 향상시키는 기술입니다. Text-to-SQL 분야에서 RAG를 활용하면 데이터베이스 스키마 정보를 효과적으로 활용하여 더 정확한 SQL 쿼리를 생성할 수 있습니다.

---

## 실습 환경 권장사항
이 노트북을 실행하기 위한 권장 환경입니다.

**Amazon SageMaker를 사용하는 경우:**
- SageMaker 이미지: `sagemaker-distribution-cpu`
- 커널: `Python 3`
- 인스턴스 타입: `ml.m5.large`

**참고:** 이 설정은 비용 효율적이면서도 충분한 성능을 제공합니다.

---

## 목차

1. [필수 라이브러리 설치](#1단계-필수-라이브러리-설치)
1. [AWS 서비스 연결 설정](#2단계-aws-서비스-연결-설정)
1. [RAG 시스템 구성요소 준비](#3단계-rag-시스템-구성요소-준비)
1. [벡터 임베딩 모델 설정](#4단계-벡터-임베딩-모델-설정)
1. [데이터베이스 스키마 분석](#5단계-데이터베이스-스키마-분석)
1. [질문 생성 및 벡터화](#6단계-질문-생성-및-벡터화)
1. [RAG 기반 SQL 생성](#7단계-rag-기반-sql-생성)
1. [실제 쿼리 실행 및 검증](#8단계-실제-쿼리-실행-및-검증)

---

## 학습 목표

이 노트북을 완료하면 다음을 할 수 있게 됩니다:

1. **RAG 시스템의 핵심 구성요소 이해**
   - 벡터 임베딩과 유사도 검색의 원리
   - 메타데이터와 자연어 질문의 매칭 방법
   - FAISS 벡터 데이터베이스 활용법

2. **실용적인 Text-to-SQL 시스템 구축**
   - 데이터베이스 스키마 정보 자동 추출
   - AI를 활용한 질문 생성 및 임베딩
   - 의미적 유사성 기반 테이블 매칭

3. **Amazon Bedrock과 AWS 서비스 통합**
   - Bedrock 임베딩 모델 활용
   - Athena 데이터베이스 연동
   - 클라우드 기반 AI 시스템 구축

## 왜 이것이 중요한가요?
- **정확성 향상**: 관련 스키마 정보를 자동으로 찾아 더 정확한 SQL 생성
- **확장성**: 수백 개의 테이블을 가진 대규모 데이터베이스에서도 효율적으로 작동
- **사용자 경험**: 비개발자도 자연어로 복잡한 데이터 분석 수행 가능

---

## RAG 기반 Text-to-SQL 시스템 아키텍처

### 기존 접근법의 한계
기존의 Text-to-SQL 방법들은 다음과 같은 문제점이 있었습니다:
- **스키마 정보 부족**: 모든 테이블 스키마를 프롬프트에 포함시키기 어려움
- **토큰 제한**: LLM의 컨텍스트 길이 제한으로 인한 정보 손실
- **비효율성**: 불필요한 테이블 정보까지 모두 전송

### RAG 기반 해결책
RAG를 활용하면 이러한 문제들을 효과적으로 해결할 수 있습니다:
- **지능형 검색**: 질문과 관련된 테이블만 선별적으로 검색
- **효율적인 활용**: 필요한 스키마 정보만 프롬프트에 포함
- **확장성**: 대규모 데이터베이스에서도 빠른 응답 시간 보장

### RAG 시스템 구축 과정 상세 설명

이 실습에서 구축할 RAG 시스템은 다음 4단계로 구성됩니다:

**1단계: 데이터베이스 메타데이터 수집**
먼저 Athena(AWS의 서버리스 쿼리 서비스)에서 TPC-DS 데이터셋의 메타데이터를 가져옵니다.
- **메타데이터란?** 데이터에 대한 데이터로, 여기서는 테이블 이름, 컬럼 이름, 데이터 타입 등의 정보
- **TPC-DS 데이터셋**: 소매업체의 판매 데이터를 모방한 표준 벤치마크 데이터셋

**2단계: AI 기반 질문 생성**
각 테이블의 스키마 정보를 바탕으로 LLM에게 "이 테이블로 답할 수 있는 자연어 질문들"을 생성하도록 요청합니다.
- 예: customer 테이블 → "어떤 고객이 가장 많이 구매했나요?", "고객 연령대별 분포는 어떻게 되나요?"

**3단계: 벡터 임베딩 및 저장**
생성된 질문들과 메타데이터를 벡터로 변환하여 FAISS 벡터 데이터베이스에 저장합니다.
- **임베딩(Embedding)**: 텍스트를 수치 벡터로 변환하여 컴퓨터가 의미를 이해할 수 있게 하는 기술
- **FAISS**: 페이스북에서 개발한 고성능 벡터 검색 라이브러리
- **의미적 유사성**: 단어의 표면적 일치가 아닌 의미적 관련성을 기준으로 유사도 측정

**4단계: 지능형 프롬프트 구성**
사용자 질문이 들어오면 벡터 검색을 통해 관련 테이블을 찾고, 해당 스키마 정보만을 포함한 효율적인 프롬프트를 구성하여 SQL을 생성합니다.

![Alt text](./content/rag.png)

### 사용할 도구들 소개

이 실습에서는 다음 라이브러리와 도구들을 사용합니다:

**핵심 라이브러리:**
- **LangChain**: AI 애플리케이션 개발을 위한 프레임워크
- **Amazon Bedrock SDK (boto3)**: AWS AI 서비스 접근을 위한 라이브러리
- **FAISS**: 고성능 벡터 유사도 검색 라이브러리
- **SQLAlchemy**: 파이썬 데이터베이스 연결 및 쿼리 실행 도구

**AWS 서비스:**
- **Amazon Bedrock**: 완전 관리형 AI 서비스 (임베딩 모델 제공)
- **Amazon Athena**: 서버리스 SQL 쿼리 서비스
- **TPC-DS on S3**: 표준 벤치마크 데이터셋

각 도구의 역할을 이해하면 전체 시스템이 어떻게 작동하는지 파악하기 쉬워집니다.

---

### 1단계: 필수 라이브러리 설치

RAG 기반 Text-to-SQL 시스템 구축을 위해 필요한 모든 라이브러리를 설치합니다.

**설치되는 라이브러리의 역할:**
- `sqlalchemy`: 데이터베이스 연결 및 쿼리 실행
- `boto3`: AWS 서비스 연결 (Bedrock, Athena)
- `langchain` & `langchain-aws`: AI 애플리케이션 개발 프레임워크
- `faiss-cpu`: 벡터 검색을 위한 고성능 라이브러리
- `PyAthena`: Amazon Athena 전용 커넥터
- `jinja2`: 프롬프트 템플릿 처리
- `jq`: JSON 데이터 처리

**설치 시 참고사항**: 
일부 의존성 충돌 경고가 나타날 수 있지만, 이번 실습에는 영향을 주지 않으므로 무시하셔도 됩니다.

In [ ]:
!python -m ensurepip --upgrade
%pip install -qU sqlalchemy
%pip install -q "boto3~=1.34" 
%pip install -qU jinja2
%pip install -qU botocore
%pip install -qU pandas
%pip install -qU PyAthena
%pip install -qU faiss-cpu
%pip install -qU langchain
%pip install -qU langchain-aws
%pip install -qU jq

---

### 2단계: 필요한 라이브러리 가져오기

RAG 기반 Text-to-SQL 시스템 구축을 위한 모든 라이브러리를 가져옵니다.

**주요 라이브러리 소개:**
- `boto3`: AWS 서비스 (Bedrock, Athena) 접근
- `langchain`: AI 애플리케이션 개발 프레임워크
- `BedrockEmbeddings`: Amazon Bedrock 임베딩 모델 인터페이스
- `FAISS`: 고성능 벡터 검색 엔진
- `SQLAlchemy`: 데이터베이스 연결 및 메타데이터 조회
- `din_sql_lib`: DIN-SQL 방법론 구현체 (SQL 생성용)
- `utilities`: 워크샵용 유틸리티 함수들

이 라이브러리들이 함께 작동하여 완전한 RAG 시스템을 구성합니다.

In [ ]:
import os
import sys
import json
from functools import partial
import json
import re

import boto3
from botocore.config import Config
from langchain.document_loaders.json_loader import JSONLoader
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain_aws import BedrockEmbeddings
from functools import reduce
from langchain.prompts import PromptTemplate
from sqlalchemy import MetaData
from sqlalchemy import create_engine

sys.path.append('../')
from libs.din_sql import din_sql_lib as dsl
import utilities as u

---

### 3단계: AWS 서비스 연결 설정

Amazon Athena와 Bedrock에 연결하기 위한 구성 정보를 설정합니다.

**설정되는 구성 정보:**
- **ATHENA_RESULTS_S3_LOCATION**: 쿼리 결과가 저장될 S3 버킷 위치
- **ATHENA_CATALOG_NAME**: 사용할 Athena 데이터 카탈로그 이름
- **DB_NAME**: 연결할 데이터베이스 이름 (tpcds1)
- **DB_FAISS_PATH**: FAISS 벡터 데이터베이스가 저장될 로컬 경로

이 정보들은 CloudFormation 스택에서 자동으로 생성되어 추출됩니다.

In [ ]:
ATHENA_RESULTS_S3_LOCATION, ATHENA_CATALOG_NAME = \
    u.extract_CF_outputs("AthenaResultsS3Location", "AthenaCatalogName")
DB_NAME = "tpcds1"
DB_FAISS_PATH = './vectorstore/db_faiss'

ATHENA_RESULTS_S3_LOCATION, ATHENA_CATALOG_NAME, DB_NAME, DB_FAISS_PATH

In [ ]:
model_id: str = "anthropic.claude-v2"
# model_id: str = "amazon.titan-tg1-large"
temperature: float = 0.2
top_k: int = 200

In [ ]:
bedrock_region = athena_region = boto3.session.Session().region_name

In [ ]:
retry_config = Config(retries={'max_attempts': 100})
session = boto3.Session(region_name=bedrock_region)
bedrock = session.client('bedrock-runtime', region_name=bedrock_region,
                         config=retry_config)

---

### 4단계: 핵심 도우미 함수 구성

RAG 시스템의 핵심 기능을 담당하는 도우미 함수들을 준비합니다.

#### Bedrock 호출 함수 설정
먼저 Amazon Bedrock과의 상호작용을 간소화하는 함수를 설정합니다. `partial` 함수를 사용하여 매번 반복되는 파라미터들을 미리 고정해둡니다.

**설정되는 파라미터들:**
- **model_id**: 사용할 AI 모델 (Claude V2)
- **temperature**: 생성 결과의 창의성 조절 (0.2 = 보수적)
- **top_k**: 토큰 선택 시 고려할 후보 수 (200개)

In [ ]:
run_bedrock = partial(u.run_bedrock_simple_prompt,
                      system_prompts=[],
                      model_id=model_id,
                      temperature=temperature,
                      top_k=top_k)

#### 질문 생성 결과 저장 함수

LLM이 각 테이블에 대해 생성한 자연어 질문들을 JSON 파일로 저장하는 함수입니다.

**이 함수의 역할:**
1. **텍스트 파싱**: LLM의 응답에서 개별 질문들을 추출
2. **데이터 구조화**: 각 질문을 테이블명, 스키마 정보와 함께 구조화
3. **JSON 저장**: 나중에 벡터화할 수 있도록 JSON 형태로 저장

**저장되는 데이터 구조:**
```json
{
  "tableName": "customer",
  "question": "어떤 고객이 가장 많이 구매했나요?",
  "tableSchema": "c_customer_sk|c_first_name|c_last_name|..."
}
```

In [ ]:
def write_questions_to_file(question_list_filename: str,
                            table_name: str,
                            table_schema, answer):
    data_list = []
    question_list_obj = answer
    questions_list = question_list_obj.splitlines()
    print(questions_list)
    # Open the file in write mode
    with open(question_list_filename, mode="w", newline="") as file:
        for question in questions_list:

            # Skip if it doesn't really have a question
            if "?" not in question:
                continue

            questionSplit = re.split(r"\d{1,5}.||. ||- ", question, maxsplit=1)
            print(questionSplit)
            question = questionSplit[1]
            data = {
                "tableName": table_name,
                "question": question,
                "tableSchema": table_schema.lstrip(" "),
            }
            data_list.append(data)

        json.dump(data_list, file)

#### 문서 처리 및 벡터화 준비 함수

JSON으로 저장된 질문 데이터를 FAISS 벡터 데이터베이스에 인덱싱하기 위해 LangChain Document 형태로 변환하는 함수들입니다.

**핵심 개념 설명:**
- **LangChain Document**: 텍스트 내용(`page_content`)과 메타데이터(`metadata`)를 함께 저장하는 데이터 구조
- **메타데이터**: 검색 시 필터링이나 정보 제공을 위한 추가 정보
- **벡터화**: 텍스트를 숫자 벡터로 변환하여 의미적 유사성 계산을 가능하게 하는 과정

이 함수들은 데이터를 벡터 검색에 최적화된 형태로 변환합니다.

In [ ]:
def create_docs_with_correct_metadata(documents):
    """
    인덱싱에 필요한 올바른 메타데이터를 가진 새로운 문서 생성
    """
    # 새로운 문서 목록을 반환할 예정
    new_docs = []

    # 각 문서에 대해
    for doc in documents:
        # 메타데이터와 내용 가져오기
        metadata = doc.metadata
        contents = json.loads(doc.page_content)

        # 추가하고자 하는 새로운 메타데이터 계산
        new_metadata = {
            "tableName": contents["tableName"],
            "question": contents["question"],
            "tableSchema": contents["tableSchema"],
        }

        # 문서의 새로운 메타데이터 출력
        # print(new_metadata)

        new_docs.append(
            Document(page_content=new_metadata["question"], metadata=new_metadata)
        )

    return new_docs

def load_json_file(filename):
    loader = JSONLoader(file_path=filename, jq_schema=".[]", text_content=False)

    # 이것은 내부 Langchain 문서 데이터 구조입니다
    docs = loader.load()
    return docs

#### 통합 테이블 처리 함수

이 함수는 RAG 시스템의 핵심으로, 각 테이블에 대해 다음 작업을 순차적으로 수행합니다:

1. **AI 질문 생성**: 테이블 스키마를 분석하여 해당 테이블로 답할 수 있는 자연어 질문들을 생성
2. **결과 저장**: 생성된 질문들을 JSON 파일로 저장
3. **벡터 변환**: 질문들을 벡터로 변환하여 FAISS 데이터베이스에 저장
4. **증분 업데이트**: 기존 벡터 DB가 있으면 새로운 테이블 정보를 추가로 병합

**사용되는 프롬프트 전략:**
- 테이블별 맞춤형 질문 생성
- 비즈니스 용어 사용 강조
- 다양한 질문 유형 (집계, 필터링, 정렬 등) 포함

In [ ]:
def add_new_table(schema, table_name,model_id,is_incremental, bedrock_embeddings):
    """
    LLM에게 테이블 스키마를 검사하고, 해당 스키마로 답할 수 있는 질문들을 생성하도록 요청한 다음,
    이러한 질문들을 파일에 저장하고, 모든 질문을 단일 벡터 DB에 로드합니다.

    :schema         : 테이블 스키마
    :table_name     : 테이블 이름
    :model_id       : 모델 ID
    :is_incremental : 증분 처리 여부
    """
    print(f"테이블 {table_name}을 스키마 {schema}와 함께 추가 중")
    prompt = f"""
    {table_name} 테이블의 다음 스키마로 답할 수 있는 고유하고 상세한 질문들의 번호가 매겨진 목록만 반환하세요:
    {schema}.
    지시사항:
        자연어 설명만 사용하세요.
        SQL을 사용하지 마세요.
        다양한 질문 목록을 생성하되, 질문들은 고유하고 상세해야 합니다.
        질문들은 이해하고 답하기 쉬운 형식이어야 합니다.
        테이블의 정보에 대해 가능한 많이 질문하세요.
        한 번에 데이터의 여러 측면에 대해 질문할 수 있습니다.
        질문은 '무엇을', '어떤', '어떻게', '언제' 또는 '할 수 있는가'로 시작해야 합니다. 변수 이름을 사용하세요.
        질문들은 관련된 비즈니스 어휘와 용어만 사용해야 합니다.
        출력에 컬럼명을 사용하지 마세요 - 관련된 자연어 설명만 사용하세요.
        숫자 값을 출력하지 마세요.
        번호가 매겨진 목록으로 시작하는 질문들을 출력하세요.

        \n 질문들: 1.
        """
    answer = run_bedrock(prompt=prompt)
    question_list_filename = f"../questionList{table_name}.json"
    print(f"질문을 {question_list_filename}에 저장 중, 스키마 {schema}, "
          f"테이블 이름 {table_name}, 답변 {answer}.\n\n")
    write_questions_to_file(question_list_filename, table_name, schema, answer)
    docs = load_json_file(question_list_filename)
    docs = create_docs_with_correct_metadata(docs)
    print(f"문서들:\n{docs}")
    new_questions = FAISS.from_documents(docs, bedrock_embeddings)
    db_exists = True if os.path.exists(f"{DB_FAISS_PATH}/index.faiss") else False
    # 새 테이블 추가
    if is_incremental and db_exists:
            question_db = FAISS.load_local(DB_FAISS_PATH, bedrock_embeddings,
                                           allow_dangerous_deserialization=True)
            question_db.merge_from(new_questions)
            question_db.save_local(DB_FAISS_PATH)
    # 처음 로드
    else:
        print(f"is_incremental이 {str(is_incremental)}로 설정되었고/또는 벡터 DB를 찾을 수 없습니다. 생성 중...")
        new_questions.save_local(DB_FAISS_PATH)

---

### 5단계: Amazon Bedrock 임베딩 모델 설정

텍스트를 벡터로 변환하는 임베딩 모델을 설정합니다.

**임베딩 모델이란?**
텍스트를 고차원 숫자 벡터로 변환하는 AI 모델입니다. 이를 통해 컴퓨터가 텍스트의 의미를 이해하고 유사성을 계산할 수 있게 됩니다.

**Amazon Bedrock 임베딩의 장점:**
- **완전 관리형**: 모델 호스팅, 스케일링을 AWS가 자동 처리
- **고성능**: 실시간 임베딩 생성 가능
- **다국어 지원**: 한국어를 포함한 다양한 언어 지원
- **보안**: AWS의 보안 정책과 통합

이 설정으로 LangChain의 BedrockEmbeddings 클래스가 초기화됩니다.

In [ ]:
bedrock_embeddings = BedrockEmbeddings(client=bedrock)

---

### 6단계: TPC-DS 데이터셋 스키마 정보 수집

Amazon Athena에서 TPC-DS 데이터셋의 모든 테이블 구조 정보를 자동으로 수집합니다.

**TPC-DS 데이터셋 소개:**
- **TPC-DS**: Transaction Processing Performance Council - Decision Support의 줄임말
- **표준 벤치마크**: 데이터 웨어하우스 성능 측정을 위한 국제 표준 데이터셋
- **실제 비즈니스 시나리오**: 소매업체의 판매, 고객, 재고 등 실제 업무를 모방
- **복잡한 관계**: 약 25개 테이블이 서로 복잡하게 연결된 구조

**메타데이터 수집 과정:**
1. SQLAlchemy를 통해 Athena 연결
2. 데이터베이스 메타데이터 자동 반영 (reflection)
3. 각 테이블의 컬럼 정보 추출
4. 파이프(|)로 구분된 문자열 형태로 변환

In [ ]:
def get_sqlalchemy_athena(database, catalog, s3stagingathena, region):
    athena_connection_str = f'awsathena+rest://:@athena.{region}.amazonaws.com:443/{database}?s3_staging_dir={s3stagingathena}&catalog_name={catalog}'
    # Athena 엔진 생성
    return create_engine(athena_connection_str)


def get_tpc_ds_dataset(database, catalog, s3stagingathena, region):
    """ 데이터베이스 스키마 반영 """

    column_table  = []
    columns_str = ''
    table_name = ''
    metadata = MetaData()
    engine = get_sqlalchemy_athena(database, catalog, s3stagingathena, region)
    metadata.reflect(bind=engine)

    # 테이블 이름 목록 가져오기
    print(metadata.tables.keys()) 

    # 테이블별로 반복
    for table in metadata.tables:
        print(f"테이블: {table}")
        table_name = table
        columns_str = ""
        print(f"스키마: {metadata.tables[table].schema}")
        print(f"컬럼들: {metadata.tables[table].columns.keys()}")
        for column in metadata.tables[table].columns.keys():
            columns_str = columns_str + f"{column}" + "|"
        column_table.append((columns_str, table_name))
    return column_table

In [ ]:
tpc_ds = get_tpc_ds_dataset(DB_NAME, ATHENA_CATALOG_NAME,
                            ATHENA_RESULTS_S3_LOCATION, athena_region)

---

### 7단계: AI 기반 질문 생성 및 벡터 임베딩

이제 핵심 단계입니다! 각 테이블에 대해 AI가 자연어 질문을 생성하고, 이를 벡터로 변환하여 검색 가능한 형태로 저장합니다.

**이 과정에서 일어나는 일:**

1. **테이블별 질문 생성**
   - 각 테이블의 스키마를 Claude에게 전달
   - "이 테이블로 답할 수 있는 자연어 질문들을 생성해줘"라고 요청
   - 비즈니스 용어를 사용한 다양한 질문들이 생성됨

2. **질문 저장 및 구조화**
   - 생성된 질문들을 JSON 파일로 저장
   - 테이블명, 스키마 정보, 질문을 함께 저장

3. **벡터 변환 및 인덱싱**
   - 각 질문을 Bedrock 임베딩 모델로 벡터화
   - FAISS 벡터 데이터베이스에 저장하여 빠른 검색 가능

**예상 소요 시간**: 약 5분 (테이블 수와 생성되는 질문 수에 따라 변동)

**생성되는 질문 예시:**
- customer 테이블: "어떤 고객이 가장 많이 구매했나요?", "고객 연령대별 분포는?"
- web_sales 테이블: "월별 웹 판매 추이는?", "가장 인기 있는 상품은?"

In [ ]:
for x in tpc_ds:
    print(x)
    schema, table_name = x
    add_new_table(
        schema=schema,
        table_name=table_name,
        model_id=model_id,
        is_incremental=True,
        bedrock_embeddings=bedrock_embeddings)
print("\n-----------------\nFinished embedding metadata")

In [ ]:
question_db = FAISS.load_local(DB_FAISS_PATH, bedrock_embeddings,
                               allow_dangerous_deserialization=True)

In [ ]:
query = "Find the top 10 customer name by total dollars spent"

---

### 8단계: 지능형 테이블 검색 및 SQL 생성

이제 구축된 RAG 시스템을 활용하여 실제로 자연어 질문을 SQL로 변환해보겠습니다!

**RAG 검색 과정:**
1. **사용자 질문 분석**: "Find the top 10 customer name by total dollars spent"
2. **의미적 검색**: 질문을 벡터로 변환하여 유사한 질문들을 FAISS에서 검색
3. **관련 테이블 식별**: 검색 결과에서 관련 테이블들과 스키마 정보 추출
4. **키워드 매칭**: 추가로 질문의 키워드와 테이블명이 일치하는 테이블도 포함

**하이브리드 검색 전략:**
- **의미적 검색**: 벡터 유사도 기반 (customer, sales 관련 질문 매칭)
- **키워드 검색**: 단어 일치 기반 (질문에 "customer"가 있으면 customer 테이블 포함)

이 두 방법을 결합하여 누락 없이 관련 테이블을 찾아냅니다.

In [ ]:
schema = {}
results_with_scores = question_db.similarity_search_with_score(query)
for doc, score in results_with_scores:
    print(doc.metadata['question'])
    schema[doc.metadata['tableName']] = doc.metadata['tableSchema']

Anthropic Claude v2 모델로 DIN_SQL 클래스를 초기화합니다.

In [ ]:
schema

In [ ]:
din_sql = dsl.DIN_SQL(bedrock_model_id=model_id)

쿼리 실행을 준비하기 위해 Athena에 연결합니다.

In [ ]:
din_sql.athena_connect(catalog_name=ATHENA_CATALOG_NAME, 
               db_name=DB_NAME, 
               s3_prefix=ATHENA_RESULTS_S3_LOCATION)

이제 질문의 어떤 단어와도 일치하는 테이블 이름이 있는 테이블 메타데이터를 `schema` 객체에 추가하여, 놓칠 수 있는 명백한 일치 항목을 포착합니다.

In [ ]:
list_tables = din_sql.find_tables(DB_NAME)
list_words = query.split(" ")

intersection = reduce(lambda acc, x: acc + [x] if x in list_words and x not in acc else acc,
                      list_tables, [])
for table in intersection:
    if table in schema:
        print("exists")
    else:
        schema_name = din_sql.get_schema(DB_NAME, table)
        schema[table] = schema_name

이제 `schema` 객체에 무엇이 들어있는지 살펴보겠습니다.

In [ ]:
schema

#### 최적화된 프롬프트 구성

검색된 스키마 정보를 바탕으로 효율적인 프롬프트를 구성합니다.

**프롬프트 설계의 핵심 원칙:**

1. **명확한 지시사항**: 무엇을 해야 하는지 단계별로 명시
2. **스키마 정보 통합**: 관련 테이블의 스키마만 선별적으로 포함
3. **실전 예시 제공**: Few-shot learning을 위한 구체적인 예시
4. **출력 형식 지정**: XML 태그를 사용한 구조화된 출력

**기존 방식과의 차이점:**
- **기존**: 모든 테이블 정보를 프롬프트에 포함 → 토큰 낭비, 혼동 가능성
- **RAG 방식**: 관련 테이블만 선별적 포함 → 효율성, 정확성 향상

이제 실제 프롬프트를 확인하고 Claude의 응답을 살펴보겠습니다.

In [ ]:
prompt_template = PromptTemplate.from_template(
    """<Instructions>
            <database_schema></database_schema> 태그 안의 데이터베이스 스키마를 읽고 다음을 수행하세요:
            이 스키마는 테이블 이름과 파이프로 구분된 스키마의 json 목록을 포함합니다:
            1. 질문에 답하기 위해 문법적으로 올바른 awsathena 쿼리를 생성하세요.
            2. 특정 테이블의 모든 컬럼을 쿼리하지 마세요. 질문과 관련된 몇 개의 컬럼만 요청하세요.
            3. 스키마 설명에서 볼 수 있는 컬럼 이름만 사용하도록 주의하세요.
            4. 존재하지 않는 컬럼을 쿼리하지 않도록 주의하세요.
            5. 어떤 컬럼이 어떤 테이블에 속하는지 주의하세요.
            6. 필요할 때 테이블 이름으로 컬럼 이름을 한정하세요. 다음 형식을 사용해야 하며, 각각 한 줄씩 작성하세요:
            7. SQL 쿼리를 <sql></sql> 태그 안에 반환하세요.
        </Instructions>

        <database_schema>{schema}</database_schema>

        <examples>
        <question>"사용자가 몇 명인가요?"</question>
        <sql>SELECT SUM(users) FROM customers</sql>

        <question>"모바일 사용자가 몇 명인가요?"</question>
        <sql>SELECT SUM(users) FROM customer WHERE source_medium='Mobile'</sql>
        </examples>

        <question>{input_question}</question>
        """)
prompt = prompt_template.format(schema=schema, input_question=query)
print(prompt)

#### Claude를 통한 SQL 생성

구성된 프롬프트를 Claude에게 전송하여 SQL 쿼리를 생성합니다.

**생성 과정:**
1. **프롬프트 전송**: 질문 + 관련 스키마 정보 + 지시사항
2. **AI 추론**: Claude가 테이블 관계를 분석하고 적절한 SQL 구성
3. **결과 추출**: XML 태그로 구분된 SQL 쿼리 추출

**예상되는 SQL 특징:**
- 적절한 JOIN 조건으로 customer와 web_sales 테이블 연결
- 고객별 총 구매액 계산을 위한 SUM 집계 함수 사용
- TOP 10 추출을 위한 ORDER BY와 LIMIT 절

In [ ]:
answer = run_bedrock(prompt=prompt)
print(answer)
sql = u.extract_tag(answer, "sql")[0]

In [ ]:
print(sql)

#### 실제 데이터 검증

생성된 SQL 쿼리를 실제 데이터베이스에서 실행하여 결과를 확인해보겠습니다.

**검증 포인트:**
1. **쿼리 실행 성공**: 문법 오류 없이 실행되는가?
2. **결과의 타당성**: 고객 이름과 구매액이 올바르게 나타나는가?
3. **순서 정확성**: 구매액 기준으로 내림차순 정렬되어 있는가?
4. **데이터 형태**: 예상한 형태의 결과가 반환되는가?

성공적으로 실행되면 RAG 기반 Text-to-SQL 시스템이 완성된 것입니다!

In [ ]:
results = din_sql.query(sql)
results